<a href="https://colab.research.google.com/github/devfull/lang/blob/main/python/colab/method_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced method chaining

SmallTalk has a separate construct for chaining methods with the `;` operator. With this feature, it is possible to send multiple messages to the same object.

`#method-chaining` `#decorator` 

## Basic chaining with `self`

Like in most common languages, basic method chaining in Python is done by returning `self`.

In [1]:
class X:

  def a(self):
    print('a')
    return self

  def b(self):
    print('b')
    return self


_ = X().a().b()

a
b


Chained methods are unable to return a meaningful value. This basic chaining strategy prevent capturing the effect of individual method calls.

In [2]:
class X:

  def a(self):
    print('#', '#', sep='\n')
    return self

  def b(self):
    print('#')
    return self


_ = X().a().b()

#
#
#


## Capturing side effect


Instance variables are used to store side effects of method calls.

In [3]:
class X:

  def __init__(self):
    self.stdout = []

  def a(self):
    self.stdout.extend([{'a':0}])
    return self

  def b(self):
    self.stdout.extend([{'b':0}])
    return self


X().a().b().stdout

[{'a': 0}, {'b': 0}]

## Running commands

The `subprocess` module can run a command described by a list of arguments. A class could have methods that aggregate thoses instructions and an additional method that run the command built. This pattern is effective when it is possible to run multiple small commands into a single one.

In [4]:
import subprocess


class Echo:

  def __init__(self):
    self.instructions = []

  def run(self):
    return subprocess.run(['echo', '-n'] + self.instructions, stdout=subprocess.PIPE, universal_newlines=True)

  def add(self,text):
    self.instructions.extend([text])
    return self


Echo().add('a').add('b').run().stdout

'a b'

When numerous methods extend the instruction list, it is convenient using a decorator for this task. All decorated methods have their return value saved and the decorator takes care of returning `self` for chaining. 

In [5]:
class Echo:
  def __init__(self):
    self.instructions = []

  def instruction(function):
    def wrapper(self, *args, **kwargs):
      self.instructions.extend(function(self, *args, **kwargs))
      return self
    return wrapper

  @instruction
  def upper(self,text):
    return text.upper()


Echo().upper('a').upper('b').instructions

['A', 'B']